University of Helsinki, Department of Mathematics and Statistics  
MAST32001 Computational Statistics I, Autumn 2018  
Antti Honkela  

# Week 1 exercises

In [ ]:
%matplotlib inline
import numpy as np
import numpy.random as npr
import matplotlib.pyplot as plt

## 1. Input ranges for overflow and underflow

i. Write a program to find the largest integer for which exp() over double precision floating point numbers (float64) returns a finite value.

ii. The logistic function $ \phi(x) = \frac{1}{1 + \exp(-x)} $ is often used to map the real line to probabilities in the range [0, 1], for example in logistic regression. For reals, the equation $ \phi(x) = 1 $ has no solution, but the same is not true for floating point numbers on a computer. Write a program to determine the smallest integer $ x $ for which $ \phi(x) =_F 1$ when using double precision floating point (float64) arithmetic.

(Note: "Write a program" means you should not try out all values manually, but program the computer to find the solution.)

In [13]:
import numpy as np
x = np.float64(1)
while(np.exp(x) != np.inf):
    if(np.exp(x+1) == np.inf):
        break
    x = x+1
print(x)

y = np.iinfo(np.int64).max

def log(y):
    z = 1/(1+np.exp(-y))*1.0
    return z

t=log(y)
print(t)
while(t != 1.0):
    y = y-1
    t = log(y)
print(y)

709.0
1.0
9223362036854775807


## 2. Numerical computation of binomial probabilities

Write a computer program to compute sums of binomial probabilities:
$$ f(l, u, n, p) = \sum_{i=l}^u \binom{n}{i} p^i (1-p)^{n-i}. $$
Use it to compute

i. $ f(0, 5, 10, 0.25) = \sum_{i=0}^{5} \binom{10}{i} 0.25^i 0.75^{10-i} $

ii. $ f(10, 20, 20, 0.25) = \sum_{i=10}^{20} \binom{20}{i} 0.25^i 0.75^{20-i} $

iii. $ f(40, 60, 100, 0.25) = \sum_{i=40}^{60} \binom{100}{i} 0.25^i 0.75^{100-i} $

iv. $ f(75, 100, 100, 0.25) = \sum_{i=75}^{100} \binom{100}{i} 0.25^i 0.75^{100-i} $


In [21]:
import scipy.special
def binprob(l,u,n,p):
    x = 0
    for i in range(u):
        x = x + scipy.special.binom(n,i)*np.power(p,i)*np.power((1-p),(n-i))
    return x

f1 = binprob(0,5,10,0.25)
f2 = binprob(10,20,20,0.25)
f3 = binprob(40,60,100,0.25)
f4 = binprob(74,100,100,0.25)

print(f1)
print(f2)
print(f3)
print(f4)

0.9218730926513672
0.9999999999990905
0.9999999999998673
1.0


## 3. Numerical evaluation of multivariate normal probabilities

Consider the $d$-variate normal distribution $\mathcal{N}(x;\; \mu, \Sigma)$ with the log-probability density
$$ \ln p(x;\; \mu, \Sigma) = -\frac{d}{2} \ln(2 \pi) - \frac{1}{2} \ln |\det \Sigma| - \frac{1}{2} (x - \mu)^T \Sigma^{-1} (x - \mu). $$
Let $\mu = \begin{pmatrix}0 \\ 0\end{pmatrix}$, $\Sigma = \begin{pmatrix}2^2 & 2\rho \\ 2\rho & 1\end{pmatrix}$.

Evaluate the following probabilities:

i. $ \ln p( \begin{pmatrix}0 \\ 0\end{pmatrix} ; \; \mu, \Sigma), $ when $\rho = 0.8$

ii. $ \ln p( \begin{pmatrix}0 \\ 0\end{pmatrix} ; \; \mu, \Sigma), $ when $\rho = 0.999$

iii. $ \ln p( \begin{pmatrix}1 \\ 1\end{pmatrix} ; \; \mu, \Sigma), $ when $\rho = 0.999$

iv. $ \ln p( \begin{pmatrix}1 \\ 1\end{pmatrix} ; \; \mu, \Sigma), $ when $\rho = -0.999$

## 4. Rejection sampling of Beta(2, 3)

Write a programme to draw samples from the Beta(2, 3) distribution
$ p(x) = 12 x (1-x)^2, \quad 0 \le x \le 1 $  
using the uniform distribution on the interval (0, 1) as the proposal distribution $q(x)$.

Remember that you need to find a constant $M$ such that $p(x) \le M q(x)$ for all $x$. Plotting the function $p(x)/q(x)$ is a good way to start looking for one.

Plot a normed histogram of your samples together with the density to check that they match.

Draw at least 10000 samples from the distribution and report your estimate of $ \mathrm{E}[x^5] $ for $ x \sim \mathrm{Beta}(2, 3)$.

NB: Please report the actual value you obtain to Moodle, not your guess of the true value. Values within the expected Monte Carlo error will be accepted as correct.

## 5. Rejection sampling of a Gaussian with Laplace

Consider the standard Laplace(0, 1) distribution defined by
$ q(x) = 1/2 \exp(- | x | ) $, and the standard normal distribution defined by
$ p(x) = \frac{1}{\sqrt{2 \pi}} \exp( - \frac{1}{2} x^2 )$.

1. Write a program to draw samples from the Laplace(0, 1) distribution. (Hint: transformation considered on Lecture 2.) Test your program by comparing the normed histogram of the samples with the density.
2. Design a method to draw samples from the standard normal using the standard Laplace distribution as the proposal. (Hint: you need to find a constant $M$ such that $p(x) \le M q(x)$ for all $x$. Plotting the function $p(x)/q(x)$ is again useful for looking for one.)
3. Use 10000 samples to compute the expectation $ E[x^4] $ for $ x $ following the standard normal distribution.
4. Assuming the underlying uniform random generator uses full 53 bits of precision of float64, what is the largest value the rejection sampling method could in theory generate? Compare this with the value for Box-Muller transform computed on Lecture 2.